# Fairness Testing and Mitigation

> _Technology is neither good nor bad; nor is it neutral._
>
>—Merlin Kranzberg's first law of technology

## Introduction—Context and Problems

The pervasiveness of automated decision-making and widespread reliance on machine-learning solutions to problems with socio-cultural ramifications make it imperative to mindfully modulate any and all research direction with due consideration. Machine learning algorithms are now playing a substantial role in deciding who we date, what movies we watch and music we listen to, how we navigate the world and understand it; but also who goes to jail and who leaves it, who is worthy of the attention of the State's security apparatus, who is worthy of a loan, a job, or even asylum (35C3 -  Computer, Die Über Asyl (Mit)Entscheiden 2018; Suresh and Guttag 2020; Mehrabi et al. 2019; Barocas, Hardt, and Narayanan 2019; Holstein et al. 2019; Friedler, Scheidegger, and Venkatasubramanian 2016). Though it has been recognized and studied as a potential source of unfairness for over 20 years (Rakova et al. 2020), algorithmic bias is still mostly either an afterthought in productionized systems or a curiosity on the fringes of a community that is mostly obsessed with State-of-the-Art (SOTA) performance.

The chasm between industry and research (Holstein et al. 2019) adds a further degree of complexity, seeing that different stakeholders often have different understandings of fairness. Even worse, machine learning practitioners themselves are unable (and perhaps duly so) to agree on a single definition (Tutorial: 21 Fairness Definitions and Their Politics 2018), as something as complex as a topic that has occupied moral philosophers for millenia is highly contextual and does not lend itself to a single mathematical definition. This seemingly self-evident hurdle is naïvely overlooked by researchers who distill fairness from a societal moral imperative into a satisfiable optimization goal, under the assumption that satisfying that goal is akin to achieving fairness (Green and Hu 2018). That said, growing awareness has reified into conferences dedicated to these important issues, such as the [FAT/ML Conference](https://www.fatml.org/) (Fairness, Accountability and Transparency in Machine Learning) as well as increased research attention and workshops at major conferences.

 The present report comes in the context of an applied data science class at the University of Leipzig where the goal was to gain practical experience in applied methodology. As such, although we do see the problematic aspect of reducing fairness testing to statistical metrics, it remains outside the scope of this report to deal with the theoretical ramifications of such an approach and invite the reader to consult Green and Hu (2018) for a thoughtful critique of the field. In what follows, we briefly discuss possible sources of bias and where they might entrench the classical machine learning pipeline, define how unfairness can be defined as a function of such bias, and finally provide a toy example of how to mitigate such bias in one's pipeline.

## Sources of Bias—Observational Fairness Setup

In order to mitigate fairness issues, we first have to set a proper context. The [Glossary of IBM's fairness toolkit](https://aif360.mybluemix.net/resources#glossary), the tools of which form the basis of the practical part of this report provide a good taxonomy which we adopt in the following elucidation of the problem setup. We will assume the setting to correspond to a classification task, where data point correspond to persons or in general entities that might be discriminated against. We will assume every data point's feature vector to include sensitive (protected) attributes such as age, gender, caste or anything that could form the basis of discrimination (See Mehrabi et al. (2019) for nuances of discrimination that are relevant to algorithmic fairness), and groups the population into groups where parity is to be expected. Finally, we assume the presence of a specific form of bias in our classification pipeline that is both unwanted and places historically privileged groups at an advantage, and by virtue (or rather vice) unprivileged groups at a systemic disadvantage. 

Understanding the many forms of bias, and being mindful of the many places they can occur within the machine learning (read decision-making) pipeline is essential. The mathematic definitions that detect the problems have to be contextualized by an awareness of where the problems might come from. Suresh and Guttag (2020) provide an extensive survey of the kinds of biases that might creep into a machine learning pipeline, and complementary to that, Olteanu et al. (2019) look into the sorts of biases that occur at the data generation process. The following two figures from the respective papers sum up their findings and provide a useful overview of the scale of the problem. The machine learning fairness survey undertaken Mehrabi et al. (2019) sums up these two aforementioned sources and consolidates them into an impressive list of 23 distinct forms of bias.  
*****
*****
  
  
![](https://www.frontiersin.org/files/Articles/456527/fdata-02-00013-HTML/image_m/fdata-02-00013-g001.jpg)
<center><i>Social Data Generation Pipeline and Sources of Bias (Olteanu et al. 2019)</i></center>


![](./images/pipeline.png)
<center><i>Machine Learning Pipeline and Sources of Bias (Suresh and Guttag 2020)</i></center>

## Measuring Bias and (Un)Fairness

Having defined our task of interest as one of classification and reviewed possible ways in which the learning task might be affected by bias, we can now quantify problematic issues that relate to sensitive attributes using the classification metrics one can derive from such tasks' confusion matrix (Depicted below). The problem becomes apparent that there are many ways to rate classifiers, with no clear a priori indication of which metric is best. That turns out to be the problem of defining fairness metrics; they tend to be both subjective and extremely contextual. Different stakeholders will want to optimize for some outcomes, which might turn out to "anti-optimize" other outcomes for other stakeholders. As such, no mathematical formalism will provide a solution: the problem, as stated before, is one of policy, and any metric will merely have to serve as a guide. In studying the infamous COMPAS recidivism-detection algorithm, Chouldechova (2016) provides a mathematical proof for what is commonly referred to as the impossibility theorem in machine learning fairness (also independently shown in Kleinberg, Mullainathan, and Raghavan (2016)), namely that one cannot simultaneoulsy guarantee equal False Positive Rates (FPR), False Negative Rates (FNR), and Positive Predictive Values (PPV) across groups.

An important distinction to be made in defining fairness metrics, is to decide whether the problem calls for group definition of fairness, or whether individual definitions of fairness might be more suited (See Dwork et al. (2011)). Up until now, we have been making the implicit assumption of group fairness  and this is the definition which we will continue using. Individual fairness calls for a different framework in which group membership is less important, and ensuring similar (subject to definition) data points get treated the same. Another definition of fairness, namely subgroup fairness, tries to merge the previous two into one framework (Mehrabi et al. 2019).



  
![](./images/confusionmatrix.png)
<center><i>Confusion Matrix and its Derivations, from <a href="https://en.wikipedia.org/wiki/Confusion_matrix">Wikipedia</a></i></center>



All the definitions of group fairness metrics one encounters in the literature, are typically a combination of this  myriad of classifier metrics and a specific context in which this particular metric was deemed needed and sufficient. This explains why there are so many different ways to assess fairness, and we can now start looking at ways to mitigate problems when they are detected.
<!-- ![](http://aequitas.dssg.io/static/images/metrictree.png)
<center><i>Fairness Metric Decision Tree from the University of Chicago</i></center> -->

## Mitigating Bias

Intervention as early as possible and as often as possible whenever bias is detected is good practice. As bias can occur at any part of the pipeline, different algorithms for bias-mitigation usually fall into one of three categories (Mehrabi et al. 2019; Bellamy et al. 2018):
* **Pre-processing:** These methods change the dataset itself by transforming and reweighting it so as to eliminate discrimination. Note that in some legal settings, using anything but raw untouched data is forbidden.
* **In-processing:** These methods inject the fairness desideratum into the learning algorithm itself via explicit goals and constraints.
* **Post-processing:** These methods treat the entire pipeline as a black box, and act on the output of the classifier, operating downstream to change the decisions in such a way so as to ensure fairness.

  
The following graphic from IBM's fairness toolkit (Bellamy et al. 2018) showcases these distinctions all the while providing different algorithms that are representative of each category:


![](https://learning.oreilly.com/library/view/ai-fairness/9781492077664/assets/aifa_0202.png)
<center><i>Bias Mitigation—When and How to Intervene</i></center>

## Bias in Practice—A Controlled Experiment in Health Policy

What follows is a fictitious experiment on very real data to try and tackle a very real problem; namely that conceiving of an allocation policy of an imaginary vaccine to try and curb the currently indomitable SARS-CoV-2 pandemic. The testbed of this exercise is that of the United States-based, statistically representative, Medical Expenditure Panel Survey (MEPS). MEPS is a yearly survey of individual families carried out by the US Department of Health. The survey is extensive and leads to a dataset with hundreds of features, including healthcare utilization. 

  
In the United States, insurance companies, in trying to keep their costs down, proactively offer care to their members. To that end, they tend to use healthcare expenditure as a proxy to predict healthcare expenses, itself a biased decision, which leads to practical racial disparities in the system. Singh and Ramamurthy (2019) study that bias and offer mitigation strategies.

  
This report takes inspiration from Singh and Ramamurthy (2019) as well as the [IBM Fairness Toolkit Medical Expenditure Tutorial](https://nbviewer.jupyter.org/github/IBM/AIF360/blob/master/examples/tutorial_medical_expenditure.ipynb) to conceive a scenario in which a government might use past healthcare expenditure as a proxy to allocate medical resources, in our case a SARS-CoV-2 vaccine. We use the IBM fairness toolkit (Bellamy et al. 2018) to measure and attempt to mitigate bias that leads to disparities.

### The MEPS Data

> _"The Medical Expenditure Panel Survey, which began in 1996, is a set of large-scale surveys of families and individuals, their medical providers (doctors, hospitals, pharmacies, etc.), and employers across the United States. The MEPS Household Component (MEPS-HC) survey collects information from families and individuals pertaining to medical expenditures, conditions, and events; demographics (e.g., age, ethnicity, and income); health insurance coverage; access to care; health status; and jobs held. Each surveyed household is interviewed five times (rounds) over a two-year period"_
![](https://raw.githubusercontent.com/HHS-AHRQ/MEPS/master/_images/panel_design.png)
<center><i>From the official <a href="https://github.com/HHS-AHRQ/MEPS">repository</a> of the MEPS survey.</i></center>

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
import pandas as pd
import numpy as np
from IPython.display import Markdown, display, HTML, IFrame
from facets_overview.generic_feature_statistics_generator import GenericFeatureStatisticsGenerator
import base64
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.explainers import MetricTextExplainer, MetricJSONExplainer
from aif360.algorithms.inprocessing import AdversarialDebiasing, PrejudiceRemover
from aif360.algorithms.preprocessing import Reweighing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import tensorflow as tf
from collections import defaultdict, OrderedDict
from functools import reduce
import json

`extract181.R`
___
```r
install.packages("foreign")
library(foreign)
setwd("/r/data/MEPS")
unzip("h181ssp.zip")
df = read.xport("h181.ssp")
write.csv(df, file="h181.csv", row.names=FALSE, quote=FALSE)
file.remove("h181.ssp")
```
___

```bash
docker run --rm --mount type=bind,source=$PWD,target=/r r-base Rscript /r/scripts/MEPS/extract181.R
docker run --rm --mount type=bind,source=$PWD,target=/r r-base Rscript /r/scripts/MEPS/extract192.R
```
___

In [5]:
%%time
df181 = pd.read_csv("data/MEPS/h181.csv")
df192 = pd.read_csv("data/MEPS/h192.csv")

df181['Race'] = df181.apply(lambda x: 'WHITE' if ((x['HISPANX']==2) and (x['RACEV2X']==1)) else 'POC', axis=1)
df192['Race'] = df192.apply(lambda x: 'WHITE' if ((x['HISPANX']==2) and (x['RACEV2X']==1)) else 'POC', axis=1)

panel19 = df181.loc[df181['PANEL'] == 19].copy()
panel20 = df181.loc[df181['PANEL'] == 20].copy()
panel21 = df192.loc[df192['PANEL'] == 21].copy()

panel19.rename(columns = {'SEX':'Sex','FTSTU53X' : 'StudentStatus', 'ACTDTY53' : 'MilitaryDuty', 'HONRDC53' : 'HonorableDischarge', 'RTHLTH53' : 'PerceivedHealth',
                        'MNHLTH53' : 'PerceivedMentalHealth', 'CHBRON53' : 'Bronchitis', 'JTPAIN53' : 'JointPain', 'PREGNT53' : 'Pregnant',
                        'WLKLIM53' : 'PhysicalLimitation', 'ACTLIM53' : 'AnyLimitation', 'SOCLIM53' : 'SocialLimitation', 'COGLIM53' : 'CognitiveLimitation',
                        'EMPST53' : 'EmploymentStatus', 'REGION53' : 'Region', 'MARRY53X' : 'MaritalStatus', 'AGE53X' : 'Age',
                        'POVCAT15' : 'Income', 'INSCOV15' : 'InsuranceCoverage', 'HIBPDX':'HighBP', 'CHDDX':'HeartDisease', 'ANGIDX':'Angina',
                        'EDUCYR':'Education', 'HIDEG':'HighestDegree',  'MIDX':'HeartAttack', 'OHRTDX':'HeartDisease2', 'STRKDX':'Stroke', 
                        'CHOLDX':'Cholesterol', 'CANCERDX':'Cancer', 'DIABDX':'Diabetes','ARTHDX':'Arthritis', 'ARTHTYPE':'ArthritisType', 'ASTHDX':'Asthma', 
                        'ADHDADDX':'ADHD', 'DFHEAR42':'HearingDifficulties','DFSEE42':'SeeingDifficulties','ADSMOK42':'Smoker', 'PHQ242':'Feelings',
                         'PCS42':'PhysicalCompontentSummary', 'MCS42':'MentalComponentSummary','K6SUM42':'Feelings2', 'EMPHDX':'Emphysema'}, inplace=True)
panel20.rename(columns = {'SEX':'Sex','FTSTU53X' : 'StudentStatus', 'ACTDTY53' : 'MilitaryDuty', 'HONRDC53' : 'HonorableDischarge', 'RTHLTH53' : 'PerceivedHealth',
                              'MNHLTH53' : 'PerceivedMentalHealth', 'CHBRON53' : 'Bronchitis', 'JTPAIN53' : 'JointPain', 'PREGNT53' : 'Pregnant',
                              'WLKLIM53' : 'PhysicalLimitation', 'ACTLIM53' : 'AnyLimitation', 'SOCLIM53' : 'SocialLimitation', 'COGLIM53' : 'CognitiveLimitation',
                              'EMPST53' : 'EmploymentStatus', 'REGION53' : 'Region', 'MARRY53X' : 'MaritalStatus', 'AGE53X' : 'Age',
                              'POVCAT15' : 'Income', 'INSCOV15' : 'InsuranceCoverage', 'HIBPDX':'HighBP', 'CHDDX':'HeartDisease', 'ANGIDX':'Angina',
                        'EDUCYR':'Education', 'HIDEG':'HighestDegree',  'MIDX':'HeartAttack', 'OHRTDX':'HeartDisease2', 'STRKDX':'Stroke', 
                        'CHOLDX':'Cholesterol', 'CANCERDX':'Cancer', 'DIABDX':'Diabetes','ARTHDX':'Arthritis', 'ARTHTYPE':'ArthritisType', 'ASTHDX':'Asthma', 
                        'ADHDADDX':'ADHD', 'DFHEAR42':'HearingDifficulties','DFSEE42':'SeeingDifficulties','ADSMOK42':'Smoker', 'PHQ242':'Feelings',
                         'PCS42':'PhysicalCompontentSummary', 'MCS42':'MentalComponentSummary','K6SUM42':'Feelings2', 'EMPHDX':'Emphysema'}, inplace=True)
panel21.rename(columns = {'SEX':'Sex','FTSTU53X' : 'StudentStatus', 'ACTDTY53' : 'MilitaryDuty', 'HONRDC53' : 'HonorableDischarge', 'RTHLTH53' : 'PerceivedHealth',
                              'MNHLTH53' : 'PerceivedMentalHealth', 'CHBRON53' : 'Bronchitis', 'JTPAIN53' : 'JointPain', 'PREGNT53' : 'Pregnant',
                              'WLKLIM53' : 'PhysicalLimitation', 'ACTLIM53' : 'AnyLimitation', 'SOCLIM53' : 'SocialLimitation', 'COGLIM53' : 'CognitiveLimitation',
                              'EMPST53' : 'EmploymentStatus', 'REGION53' : 'Region', 'MARRY53X' : 'MaritalStatus', 'AGE53X' : 'Age',
                              'POVCAT16' : 'Income', 'INSCOV16' : 'InsuranceCoverage', 'HIBPDX':'HighBP', 'CHDDX':'HeartDisease', 'ANGIDX':'Angina',
                        'EDUCYR':'Education', 'HIDEG':'HighestDegree',  'MIDX':'HeartAttack', 'OHRTDX':'HeartDisease2', 'STRKDX':'Stroke', 
                        'CHOLDX':'Cholesterol', 'CANCERDX':'Cancer', 'DIABDX':'Diabetes','ARTHDX':'Arthritis', 'ARTHTYPE':'ArthritisType', 'ASTHDX':'Asthma', 
                        'ADHDADDX':'ADHD', 'DFHEAR42':'HearingDifficulties','DFSEE42':'SeeingDifficulties','ADSMOK42':'Smoker', 'PHQ242':'Feelings',
                         'PCS42':'PhysicalCompontentSummary', 'MCS42':'MentalComponentSummary','K6SUM42':'Feelings2', 'EMPHDX':'Emphysema'}, inplace=True)

panel19 = panel19.loc[(panel19['Region']>=0)&(panel19['Age']>=0)&(panel19['MaritalStatus']>=0)&(panel19['Asthma']>=0)].copy()
panel20 = panel20.loc[(panel20['Region']>=0)&(panel20['Age']>=0)&(panel20['MaritalStatus']>=0)&(panel20['Asthma']>=0)].copy()
panel21 = panel21.loc[(panel21['Region']>=0)&(panel21['Age']>=0)&(panel21['MaritalStatus']>=0)&(panel21['Asthma']>=0)].copy()

panel19.loc[:,'Utilization'] = panel19.apply(lambda x: x['OBTOTV15'] + x['OPTOTV15'] + x['ERTOT15'] + x['IPNGTD15'] + x['HHTOTD15'], axis=1)
panel20.loc[:,'Utilization'] = panel20.apply(lambda x: x['OBTOTV15'] + x['OPTOTV15'] + x['ERTOT15'] + x['IPNGTD15'] + x['HHTOTD15'], axis=1)
panel21.loc[:,'Utilization'] = panel21.apply(lambda x: x['OBTOTV16'] + x['OPTOTV16'] + x['ERTOT16'] + x['IPNGTD16'] + x['HHTOTD16'], axis=1)

outlier_filter = ['StudentStatus','MilitaryDuty','HonorableDischarge','PerceivedHealth','PerceivedMentalHealth','HighBP','HeartDisease','Angina','Education','HighestDegree',
                             'HeartAttack','HeartDisease2','Stroke','Emphysema','Bronchitis','Cholesterol','Cancer','Diabetes',
                             'JointPain','Arthritis','ArthritisType','Asthma','ADHD','Pregnant','PhysicalLimitation',
                             'AnyLimitation','SocialLimitation','CognitiveLimitation','HearingDifficulties','SeeingDifficulties','Smoker',
                             'Feelings','EmploymentStatus','Income','InsuranceCoverage']

panel19 = panel19.loc[(panel19[outlier_filter]>=-1).all(axis=1), :].copy()
panel20 = panel20.loc[(panel20[outlier_filter]>=-1).all(axis=1), :].copy()
panel21 = panel21.loc[(panel21[outlier_filter]>=-1).all(axis=1), :].copy()

panel19.loc[:,'Vaccine'] = np.where(panel19['Utilization']>=8, 1.0, 0.0)
panel20.loc[:,'Vaccine'] = np.where(panel20['Utilization']>=8, 1.0, 0.0)
panel21.loc[:,'Vaccine'] = np.where(panel21['Utilization']>=8, 1.0, 0.0)

categorical = ['Region','Sex','MaritalStatus','StudentStatus','MilitaryDuty','HonorableDischarge','PerceivedHealth','PerceivedMentalHealth','HighBP','HeartDisease','Angina','HeartAttack','HeartDisease2',
            'Stroke','Emphysema','Bronchitis','Cholesterol','Cancer','Diabetes','JointPain','Arthritis','ArthritisType','Asthma','ADHD','Pregnant',
            'PhysicalLimitation','AnyLimitation','SocialLimitation','CognitiveLimitation','HearingDifficulties','SeeingDifficulties','Smoker','Feelings','EmploymentStatus','Income','InsuranceCoverage']

keep = ['Region','Age','Sex','Race','MaritalStatus','StudentStatus','MilitaryDuty','HonorableDischarge','PerceivedHealth','PerceivedMentalHealth','HighBP','HeartDisease','Angina','HeartAttack','HeartDisease2',
            'Stroke','Emphysema','Bronchitis','Cholesterol','Cancer','Diabetes','JointPain','Arthritis','ArthritisType','Asthma','ADHD','Pregnant',
           'PhysicalLimitation','AnyLimitation','SocialLimitation','CognitiveLimitation','HearingDifficulties','SeeingDifficulties','Smoker','PhysicalCompontentSummary', 'MentalComponentSummary','Feelings2','Feelings','EmploymentStatus','Income','InsuranceCoverage','Vaccine']

dataset19 = StandardDataset(df=panel19, label_name='Vaccine', favorable_classes=[1.0], protected_attribute_names=['Race'], privileged_classes=[['WHITE']], 
                            instance_weights_name='PERWT15F', categorical_features=categorical, features_to_keep=keep+['PERWT15F'])

dataset20 = StandardDataset(df=panel20, label_name='Vaccine', favorable_classes=[1.0], protected_attribute_names=['Race'], privileged_classes=[['WHITE']], 
                            instance_weights_name='PERWT15F', categorical_features=categorical, features_to_keep=keep+['PERWT15F'])

dataset21 = StandardDataset(df=panel21, label_name='Vaccine', favorable_classes=[1.0], protected_attribute_names=['Race'], privileged_classes=[['WHITE']], 
                            instance_weights_name='PERWT16F', categorical_features=categorical, features_to_keep=keep+['PERWT16F'])

CPU times: user 17.4 s, sys: 1.76 s, total: 19.1 s
Wall time: 19.2 s


In [203]:
proto = GenericFeatureStatisticsGenerator().ProtoFromDataFrames([{'name': 'Panel19', 'table': panel19[categorical].astype(str)}])
protostr = base64.b64encode(proto.SerializeToString()).decode("utf-8")
HTML_TEMPLATE = """
        <script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
html_str = HTML_TEMPLATE.format(protostr=protostr)
with open("facets/panel19overview.html",'w') as fo:
    fo.write(html_str)

### Interactive Dataset Explorer

In [8]:
display(IFrame("facets/panel19overview.html", width=2000,height=1000))

In [ ]:
df = panel19[keep]
sprite_size = 32 if len(df.index)>50000 else 64

jsonstr = df.to_json(orient='records') 
HTML_TEMPLATE = """
        <script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-dive sprite-image-width="{sprite_size}" sprite-image-height="{sprite_size}" id="elem" height="900"></facets-dive>
        <script>
          document.querySelector("#elem").data = {jsonstr};
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=jsonstr, sprite_size=sprite_size)
with open("facets/panel19dive.html",'w') as fo:
    fo.write(html)

In [11]:
display(IFrame("facets/panel19dive.html", width=2000,height=950))

### Fit Logistic Regression and compute metrics:

In [6]:
dataset19_train, dataset19_val, dataset19_test = dataset19.split([0.5, 0.8], shuffle=True)
pipeline19 = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', random_state=42))
log_reg_19 = pipeline19.fit(dataset19_train.features, dataset19_train.labels.ravel(), **{'logisticregression__sample_weight': dataset19_train.instance_weights})
y_prob_val_19 = pipeline19.predict_proba(dataset19_val.features)

In [7]:
metrics_19_val = defaultdict(list)
for threshold in np.linspace(0.01, 0.5, 500):
    y_hat_val = np.float64(y_prob_val_19[:, 1] > threshold)
    ds_ = dataset19_val.copy()
    ds_.labels = y_hat_val
    metric_dataset19_val = ClassificationMetric(dataset19_val, ds_, unprivileged_groups=[{'Race': 0.0}], privileged_groups=[{'Race': 1.0}])
    tp=metric_dataset19_val.true_positive_rate()
    tn=metric_dataset19_val.true_negative_rate()
    bal_acc= (tp+tn)/2
    metrics_19_val['Threshold'].append(threshold)
    metrics_19_val['Balanced_Accuracy'].append(bal_acc) 
    metrics_19_val['True_Positive_Rate'].append(tp) 
    metrics_19_val['True_Negative_Rate'].append(tn)
    metrics_19_val['Average_Odds_Difference'].append(metric_dataset19_val.average_odds_difference())
    metrics_19_val['Disparate_Impact'].append(metric_dataset19_val.disparate_impact())
    metrics_19_val['Statistical_Parity_Difference'].append(metric_dataset19_val.statistical_parity_difference())
    metrics_19_val['Equal_Opportunity_Difference'].append(metric_dataset19_val.equal_opportunity_difference())
    metrics_19_val['Theil_Index'].append(metric_dataset19_val.theil_index())

In [8]:
best_threshold = metrics_19_val['Threshold'][np.argmax(metrics_19_val['Balanced_Accuracy'])]
y_hat_val = np.float64(y_prob_val_19[:, 1] > best_threshold)
ds_ = dataset19_val.copy()
ds_.labels = y_hat_val
metric_for_best_threshold19 = ClassificationMetric(dataset19_val, ds_, unprivileged_groups=[{'Race': 0.0}], privileged_groups=[{'Race': 1.0}])
json_explainer = MetricJSONExplainer(metric_for_best_threshold19)
text_explainer = MetricTextExplainer(metric_for_best_threshold19)

In [9]:
print(json.dumps(json.loads(json_explainer.disparate_impact(), object_pairs_hook=OrderedDict), indent=10))

{
          "metric": "Disparate Impact",
          "message": "Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.41711314452838666",
          "numPositivePredictionsUnprivileged": 2842238.767988,
          "numUnprivileged": 16705464.576229,
          "numPositivePredictionsPrivileged": 8710516.913715,
          "numPrivileged": 26380364.489999004,
          "description": "Computed as the ratio of rate of favorable outcome for the unprivileged group to that of the privileged group.",
          "ideal": "The ideal value of this metric is 1.0 A value < 1 implies higher benefit for the privileged group and a value >1 implies a higher benefit for the unprivileged group."
}


In [10]:
print(json.dumps(json.loads(json_explainer.theil_index(), object_pairs_hook=OrderedDict), indent=10))

{
          "metric": "Theil Index",
          "message": "Theil index (generalized entropy index with alpha = 1): 0.10673063853868728",
          "description": "Computed as the generalized entropy of benefit for all individuals in the dataset, with alpha = 1. It measures the inequality in benefit allocation for individuals.",
          "ideal": "A value of 0 implies perfect fairness."
}


In [184]:
print(json.dumps(json.loads(json_explainer.equal_opportunity_difference(), object_pairs_hook=OrderedDict), indent=10))

{
          "metric": "True Positive Rate Difference",
          "message": "True positive rate difference (true positive rate on unprivileged instances - true positive rate on privileged instances): -0.1396753669721943",
          "numTruePositivesUnprivileged": 1763331.1644010001,
          "numPositivesUnprivileged": 2813481.0430280003,
          "numTruePositivesPrivileged": 6407872.478816999,
          "numPositivesPrivileged": 8360796.08152,
          "description": "This metric is computed as the difference of true positive rates between the unprivileged and the privileged groups.  The true positive rate is the ratio of true positives to the total number of actual positives for a given group.",
          "ideal": "The ideal value is 0. A value of < 0 implies higher benefit for the privileged group and a value > 0 implies higher benefit for the unprivileged group."
}


In [185]:
print(json.dumps(json.loads(json_explainer.statistical_parity_difference(), object_pairs_hook=OrderedDict), indent=10))

{
          "metric": "Statistical Parity Difference",
          "message": "Statistical parity difference (probability of favorable outcome for unprivileged instances - probability of favorable outcome for privileged instances): -0.25451713366606493",
          "numPositivesUnprivileged": 2813481.0430280003,
          "numInstancesUnprivileged": 17245903.276352998,
          "numPositivesPrivileged": 8360796.08152,
          "numInstancesPrivileged": 25099245.996873997,
          "description": "Computed as the difference of the rate of favorable outcomes received by the unprivileged group to the privileged group.",
          "ideal": " The ideal value of this metric is 0"
}


In [126]:
metrics_19_val['Average_Odds_Difference'][99]

-0.23381977977374965

In [118]:
metric_dataset19_val.theil_index()

0.08364354219673947

In [25]:
panel19['Vaccine'].value_counts()

0.0    12509
1.0     3321
Name: Vaccine, dtype: int64

In [65]:
metric_dataset19_train = BinaryLabelDatasetMetric(dataset19_train,unprivileged_groups=[{'Race': 0.0}], privileged_groups=[{'Race': 1.0}])
explainer_dataset19_train = MetricTextExplainer(metric_dataset19_train)

print(explainer_dataset19_train.disparate_impact())

Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.5162450090494392


In [59]:
explainer_orig_panel19_train.mean_difference()

'Mean difference (mean label value on unprivileged instances - mean label value on privileged instances): -0.15795562961249188'

In [60]:
explainer_orig_panel19_train.statistical_parity_difference()

'Statistical parity difference (probability of favorable outcome for unprivileged instances - probability of favorable outcome for privileged instances): -0.15795562961249188'

## An example of bias-mitigation through preprocessing: Reweighing

### Reweigh the dataset, mindfully of the unprivileged groups, and repeat process:

In [219]:
Reweigher = Reweighing(unprivileged_groups=[{'Race': 0.0}], privileged_groups=[{'Race': 1.0}])
dataset19_train_reweighed = Reweigher.fit_transform(dataset19_train)

In [220]:
pipeline19_rw = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', random_state=42))
log_reg_19_rw = pipeline19_rw.fit(dataset19_train_reweighed.features, dataset19_train_reweighed.labels.ravel(), **{'logisticregression__sample_weight': dataset19_train_reweighed.instance_weights})
y_prob_val_19_rw = pipeline19_rw.predict_proba(dataset19_val.features)

In [222]:
metrics_19_val_rw = defaultdict(list)
for threshold in np.linspace(0.01, 0.5, 500):
    y_hat_val_rw = np.float64(y_prob_val_19_rw[:, 1] > threshold)
    ds_ = dataset19_val.copy()
    ds_.labels = y_hat_val
    metric_dataset19_val = ClassificationMetric(dataset19_val, ds_, unprivileged_groups=[{'Race': 0.0}], privileged_groups=[{'Race': 1.0}])
    tp=metric_dataset19_val.true_positive_rate()
    tn=metric_dataset19_val.true_negative_rate()
    bal_acc= (tp+tn)/2
    metrics_19_val_rw['Threshold'].append(threshold)
    metrics_19_val_rw['Balanced_Accuracy'].append(bal_acc) 
    metrics_19_val_rw['True_Positive_Rate'].append(tp) 
    metrics_19_val_rw['True_Negative_Rate'].append(tn)
    metrics_19_val_rw['Average_Odds_Difference'].append(metric_dataset19_val.average_odds_difference())
    metrics_19_val_rw['Disparate_Impact'].append(metric_dataset19_val.disparate_impact())
    metrics_19_val_rw['Statistical_Parity_Difference'].append(metric_dataset19_val.statistical_parity_difference())
    metrics_19_val_rw['Equal_Opportunity_Difference'].append(metric_dataset19_val.equal_opportunity_difference())
    metrics_19_val_rw['Theil_Index'].append(metric_dataset19_val.theil_index())

In [224]:
best_threshold = metrics_19_val_rw['Threshold'][np.argmax(metrics_19_val['Balanced_Accuracy'])]
y_hat_val = np.float64(y_prob_val_19_rw[:, 1] > best_threshold)
ds_ = dataset19_val.copy()
ds_.labels = y_hat_val
metric_for_best_threshold19 = ClassificationMetric(dataset19_val, ds_, unprivileged_groups=[{'Race': 0.0}], privileged_groups=[{'Race': 1.0}])
json_explainer = MetricJSONExplainer(metric_for_best_threshold19)
text_explainer = MetricTextExplainer(metric_for_best_threshold19)

In [225]:
print(json.dumps(json.loads(json_explainer.disparate_impact(), object_pairs_hook=OrderedDict), indent=10))

{
          "metric": "Disparate Impact",
          "message": "Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.8018666612615467",
          "numPositivePredictionsUnprivileged": 2813481.0430280003,
          "numUnprivileged": 17245903.276352998,
          "numPositivePredictionsPrivileged": 8360796.08152,
          "numPrivileged": 25099245.996873997,
          "description": "Computed as the ratio of rate of favorable outcome for the unprivileged group to that of the privileged group.",
          "ideal": "The ideal value of this metric is 1.0 A value < 1 implies higher benefit for the privileged group and a value >1 implies a higher benefit for the unprivileged group."
}


In [227]:
print(json.dumps(json.loads(json_explainer.theil_index(), object_pairs_hook=OrderedDict), indent=10))

{
          "metric": "Theil Index",
          "message": "Theil index (generalized entropy index with alpha = 1): 0.11041818834729401",
          "description": "Computed as the generalized entropy of benefit for all individuals in the dataset, with alpha = 1. It measures the inequality in benefit allocation for individuals.",
          "ideal": "A value of 0 implies perfect fairness."
}


In [228]:
print(json.dumps(json.loads(json_explainer.equal_opportunity_difference(), object_pairs_hook=OrderedDict), indent=10))

{
          "metric": "True Positive Rate Difference",
          "message": "True positive rate difference (true positive rate on unprivileged instances - true positive rate on privileged instances): 0.01961871612076993",
          "numTruePositivesUnprivileged": 2018228.904842,
          "numPositivesUnprivileged": 2813481.0430280003,
          "numTruePositivesPrivileged": 5833524.435001,
          "numPositivesPrivileged": 8360796.08152,
          "description": "This metric is computed as the difference of true positive rates between the unprivileged and the privileged groups.  The true positive rate is the ratio of true positives to the total number of actual positives for a given group.",
          "ideal": "The ideal value is 0. A value of < 0 implies higher benefit for the privileged group and a value > 0 implies higher benefit for the unprivileged group."
}


In [229]:
print(json.dumps(json.loads(json_explainer.statistical_parity_difference(), object_pairs_hook=OrderedDict), indent=10))

{
          "metric": "Statistical Parity Difference",
          "message": "Statistical parity difference (probability of favorable outcome for unprivileged instances - probability of favorable outcome for privileged instances): -0.07739540128322908",
          "numPositivesUnprivileged": 2813481.0430280003,
          "numInstancesUnprivileged": 17245903.276352998,
          "numPositivesPrivileged": 8360796.08152,
          "numInstancesPrivileged": 25099245.996873997,
          "description": "Computed as the difference of the rate of favorable outcomes received by the unprivileged group to the privileged group.",
          "ideal": " The ideal value of this metric is 0"
}


## Conclusion


There are many abstractions to keep track of when wanting to quantify bias and unfairness in a machine learning pipeline; some of them at odds with each other. Surveying the literature, one can't help but get the impression that multidisciplinarity is direly required to advance the field and rid it of the bitter aftertaste of technological solutionism (Morozov 2014). Social scientists and moral philosophers have been wrestling with these problems for long, and it is simplistic to think one can reduce these efforts to a loss function.


That said, it can only be helpful in practice to be mindful of the different stages of bias injection and remedy those by trying out different things and operating the machine learning pipeline as the endless loop it is best to operate it within. Frameworks such as IBM's Fairness toolkit provide a powerful tool for both practitioners who want to deploy datasets and models, as well as researchers who want to study the harfmul effects these artifacts can have on the world.

## Bibliography


35C3 -  Computer, Die Über Asyl (Mit)Entscheiden. 2018. https://www.youtube.com/watch?v=My-Xxq1K3oA.


Barocas, Solon, Moritz Hardt, and Arvind Narayanan. 2019. Fairness and Machine Learning. fairmlbook.org.


Bellamy, Rachel K. E., Kuntal Dey, Michael Hind, Samuel C. Hoffman, Stephanie Houde, Kalapriya Kannan, Pranay 


Lohia, et al. 2018. “AI Fairness 360: An Extensible Toolkit for Detecting, Understanding, and Mitigating Unwanted 


Algorithmic Bias.” ArXiv:1810.01943 [Cs], October. http://arxiv.org/abs/1810.01943.


Chouldechova, Alexandra. 2016. “Fair Prediction with Disparate Impact: A Study of Bias in Recidivism Prediction Instruments.” ArXiv:1610.07524 [Cs, Stat], October. http://arxiv.org/abs/1610.07524.


Dwork, Cynthia, Moritz Hardt, Toniann Pitassi, Omer Reingold, and Rich Zemel. 2011. “Fairness Through Awareness.” ArXiv:1104.3913 [Cs], November. http://arxiv.org/abs/1104.3913.


“Friedler et al. - 2016 - On the (Im)Possibility of Fairness.Pdf.” n.d. Accessed October 31, 2020. https://arxiv.org/pdf/1609.07236.pdf.


Friedler, Sorelle A., Carlos Scheidegger, and Suresh Venkatasubramanian. 2016. “On the (Im)Possibility of Fairness.” ArXiv:1609.07236 [Cs, Stat], September. http://arxiv.org/abs/1609.07236.


Green, Ben, and Lily Hu. 2018. “The Myth in the Methodology: Towards a Recontextualization of Fairness in Machine Learning.” Stockholm, Sweden. 2018.


Holstein, Kenneth, Jennifer Wortman Vaughan, Hal Daumé III, Miro Dudík, and Hanna Wallach. 2019. “Improving Fairness in Machine Learning Systems: What Do Industry Practitioners Need?” Proceedings of the 2019 CHI Conference on Human Factors in Computing Systems  - CHI ’19, 1–16. https://doi.org/10.1145/3290605.3300830.


Kleinberg, Jon, Sendhil Mullainathan, and Manish Raghavan. 2016. “Inherent Trade-Offs in the Fair Determination of Risk Scores.” ArXiv:1609.05807 [Cs, Stat], November. http://arxiv.org/abs/1609.05807.


Mehrabi, Ninareh, Fred Morstatter, Nripsuta Saxena, Kristina Lerman, and Aram Galstyan. 2019. “A Survey on Bias and Fairness in Machine Learning.” ArXiv:1908.09635 [Cs], September. http://arxiv.org/abs/1908.09635.


Morozov, Evgeny. 2014. To Save Everything, Click Here: The Folly of Technological Solutionism. Reprint edition. New York, NY: PublicAffairs.


Olteanu, Alexandra, Carlos Castillo, Fernando Diaz, and Emre Kıcıman. 2019. “Social Data: Biases, Methodological Pitfalls, and Ethical Boundaries.” Frontiers in Big Data 2. https://doi.org/10.3389/fdata.2019.00013.


Rakova, Bogdana, Jingying Yang, Henriette Cramer, and Rumman Chowdhury. 2020. “Where Responsible AI Meets Reality: Practitioner Perspectives on Enablers for Shifting Organizational Practices.” ArXiv:2006.12358 [Cs], July. 
http://arxiv.org/abs/2006.12358.


Singh, Moninder, and Karthikeyan Natesan Ramamurthy. 2019. “Understanding Racial Bias in Health Using the Medical Expenditure Panel Survey Data.” ArXiv:1911.01509 [Cs, Stat], November. http://arxiv.org/abs/1911.01509.


Suresh, Harini, and John V. Guttag. 2020. “A Framework for Understanding Unintended Consequences of Machine Learning.” ArXiv:1901.10002 [Cs, Stat], February. http://arxiv.org/abs/1901.10002.


Tutorial: 21 Fairness Definitions and Their Politics. 2018. https://www.youtube.com/watch?v=jIXIuYdnyyk.